In [1]:
from dataclasses import dataclass, field
from typing import List

@dataclass
class Orbiter:
    name: str = 'COM'
    # Seems to be a Jupyter issue where I can't have these
    # parameters use Orbiter unless the block has already run
    # correctly once. Issue only crops up on kernel restart.
    # Changed to object to fix, doesn't matter for this.
    parent: object = None
    children: List[object] = field(default_factory=list)
    num_orbits: int = field(init=False)
        
    def __repr__(self):
        if self.parent:
            return f'{self.name}({self.parent.name})'
        else:
            return f'{self.name}()'
        
    def __post_init__(self):
        self.update_orbits()
        if self.parent:
            self.parent.add_child(self)
            
    def add_parent(self, parent):
        self.parent = parent
        self.parent.add_child(self)
        self.update_orbits()
    
    def add_child(self, child):
        self.children.append(child)
    
    def update_orbits(self):
        """Update the number of orbits then propagate down to children."""
        if self.parent:
            self.num_orbits = self.parent.num_orbits + 1
            for child in self.children:
                child.update_orbits()
        else:
            self.num_orbits = 0

In [2]:
# Test basic use
com = Orbiter('COM', None)
a = Orbiter('A', com)
b = Orbiter('B', a)
c = Orbiter('C', b)
d = Orbiter('D', a)

for orb in [com, a, b, c, d]:
    print(f'{orb} = {orb.num_orbits}')

COM() = 0
A(COM) = 1
B(A) = 2
C(B) = 3
D(A) = 2


In [3]:
# Test parent insertion
a = Orbiter('A')
b = Orbiter('B', a)
c = Orbiter('C', b)
d = Orbiter('D', a)
com = Orbiter()
a.add_parent(com)
for orb in [com, a, b, c, d]:
    print(f'{orb} = {orb.num_orbits}')

COM() = 0
A(COM) = 1
B(A) = 2
C(B) = 3
D(A) = 2


In [4]:
def parse_input(str_in):
    # 'A)B\n B)C\n C)D\n' -> [['A', 'B'], ['B', 'C'], ['C', 'D'], ['']]
    nodes = [line.split(')') for line in str_in.split('\n')]

    # Deal with the blank last item due to the newline at file end
    if nodes[-1] == ['']:
        nodes.pop()
    
    # For each node, add the parent to the orbiters dict if it doesn't already exist,
    # then add the child. 
    orbiters = {}
    for parent_str, child_str in nodes:
        if parent_str not in orbiters:
            parent = Orbiter(name=parent_str)
            orbiters[parent_str] = parent
        else:
            parent = orbiters[parent_str]
            
        if child_str not in orbiters:
            child = Orbiter(name=child_str, parent=parent)
            orbiters[child_str] = child
        else:
            orbiters[child_str].add_parent(parent)
    return orbiters

In [5]:
# Test input parsing on the example
test_str = '''COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
'''
orbs = parse_input(test_str)
print(orbs)

{'COM': COM(), 'B': B(COM), 'C': C(B), 'D': D(C), 'E': E(D), 'F': F(E), 'G': G(B), 'H': H(G), 'I': I(D), 'J': J(E), 'K': K(J), 'L': L(K)}


In [6]:
# Example solution, correct answer is 42
print(sum(orb.num_orbits for orb in orbs.values()))

42


In [7]:
# Same example but with mixed up entries.
# Should still have 42 orbits.
test_str = '''B)G
C)D
K)L
COM)B
D)E
E)F
B)C
G)H
B)G
D)I
J)K
E)J
G)H
'''
orbs = parse_input(test_str)
print(orbs)
print([f'{orb.name}:{orb.num_orbits}' for orb in orbs.values()])
print(sum(orb.num_orbits for orb in orbs.values()))

{'B': B(COM), 'G': G(B), 'C': C(B), 'D': D(C), 'K': K(J), 'L': L(K), 'COM': COM(), 'E': E(D), 'F': F(E), 'H': H(G), 'I': I(D), 'J': J(E)}
['B:1', 'G:2', 'C:2', 'D:3', 'K:6', 'L:7', 'COM:0', 'E:4', 'F:5', 'H:3', 'I:4', 'J:5']
42


In [8]:
# Part 1 solution
with open('day6_input.txt', 'r') as infile:
    in_str = infile.read()

orbs = parse_input(in_str)
print(sum(orb.num_orbits for orb in orbs.values()))

110190
